# 도로 검출

## 파라미터 : 
*  epoch : 30
* beta1: 0.9, beta2 : 0.999 
* learning rate : 0.0001

## 라벨링 방식 : 흑백 채우기 

In [2]:
!pip install segmentation_models

     |████████████████████████████████| 51kB 5.6MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import math
import os
import glob
import json
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import re
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import tensorflow as tf

from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt

In [5]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [6]:
!ls
!ls /content/drive/MyDrive/SIA/colab_code/lhk

drive  sample_data
0603_bnr_label01_v1	      0603_roads_label01_v1
0603_bnr_label01_v1.h5	      0603_roads_label01_v1.h5
0603_buildings_label01_v1     binary_buildings_label01_v1.h5
0603_buildings_label01_v1.h5  bnr_label01_v1.h5


In [7]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 1
LR = 0.0001
EPOCHS = 30
preprocess_input = sm.get_preprocessing(BACKBONE)

빌딩이냐 도로만 바꾸면됨 

In [8]:
train_input_DIR =  '/content/drive/MyDrive/SIA/roads/train/raw_image/' #input 경로

train_label_DIR = '/content/drive/MyDrive/SIA/roads/train/new_label/' #label 경로 

val_input_DIR =  '/content/drive/MyDrive/SIA/roads/val/raw_image/' #input 경로

val_label_DIR = '/content/drive/MyDrive/SIA/roads/val/new_label/' #label 경로 

In [9]:
train_input = glob.glob(train_input_DIR  + '*.png')
train_label  = glob.glob(train_label_DIR + '*.png')

val_input = glob.glob(val_input_DIR  + '*.png')
val_label  = glob.glob(val_label_DIR + '*.png')


train_input.sort()
train_label.sort()
val_input.sort()
val_label.sort()

In [10]:
CLASSES = ['roads']

In [11]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset_bi:
    CLASSES =  ['roads','buildings']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = [i.split('/')[-1] for i in images_dir]
        self.images_fps = images_dir
        self.masks_fps = masks_dir
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        mask = mask.reshape(1024,1024,-1)

        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            # 둘다 1일 경우..
            background = 255 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
            mask = np.where(mask == 255, mask, 0)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
#         image = image/255         
        mask = mask/255
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [12]:
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()


    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
       
        return tuple(batch)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)   

In [13]:
import albumentations as A

In [14]:
def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(1024, 1024)
    ]
    return A.Compose(test_transform)


In [15]:
def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [16]:
train_dataset = Dataset_bi(
    train_input,train_label 
    ,
    classes=CLASSES,
#         preprocessing=get_preprocessing(preprocess_input),
)

In [17]:
valid_dataset = Dataset_bi(
    val_input, val_label,
    classes=CLASSES,    
#     augmentation=get_validation_augmentation(),
#         preprocessing=get_preprocessing(preprocess_input)

)


## 모델 설계

In [18]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

44113920/44107200 [==============================] - 1s 0us/step


In [19]:
# define optomizer
optim = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)


# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [20]:
train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

In [21]:
# assert train_dataloader[0][0].shape == (BATCH_SIZE, 1024, 1024, 3)
# assert train_dataloader[0][1].shape == (BATCH_SIZE, 1024, 1024, n_classes)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/SIA/colab_code/lhk/0603_roads_label01_v1.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

In [22]:
model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f23635c9c90>>

In [ ]:
# train model
history = model.fit_generator(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=30, 
     callbacks=callbacks,      validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader) ,verbose = 1 
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
 244/1144 [=====>........................] - ETA: 17:45 - loss: 0.6275 - iou_score: 0.3776 - f1-score: 0.5225

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 8))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## 모델 평가

In [ ]:
test_dataset = Dataset_bi(
    val_input, val_label,
    classes=CLASSES,    
#     augmentation=get_validation_augmentation(),
#         preprocessing=get_preprocessing(preprocess_input)

)
test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

In [ ]:

# load best weights
model.load_weights('/content/drive/MyDrive/SIA/colab_code/lhk/0603_roads_label01_v1.h5')


In [ ]:
scores = model.evaluate_generator(test_dataloader)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image)
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask.squeeze(),
        pr_mask=pr_mask.squeeze(),
    )

In [ ]:
n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image)
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask.squeeze(),
        pr_mask=pr_mask.squeeze(),
    )